**NOTE:** 
- the complete execution of this notebook takes about 20 hours on Google Colab
- the train test split is deliberately included in each cell so that they can be executed independently
- I did not define a lot of functions because each cell is only about 60 lines long and defining functions in this case makes is harder to follow the data flow and in my opinion decreases readability, adding cognitive overhead




#Imports and function definitions

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statistics import mean
from sklearn import neighbors
from sklearn.model_selection import train_test_split, KFold
from sklearn.datasets import load_wine
from sklearn.preprocessing import scale, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense
from tensorflow.random import set_seed
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

# Fix the seed for reproducibility:
np.random.seed(666)
set_seed(666)

### Normalize features, transform labels into a one-hot encoding vector:
def preprocess_data(X, y, num_classes):
    scaledX = X/255
    oneHoty = to_categorical(y, num_classes=num_classes)
    return scaledX, oneHoty

def build_MLP(num_classes, nodes, layers, activation_input='relu'):
    MLP = Sequential()
    MLP.add(Flatten())
    # Hidden layers (fully connected):
    for i in range(0,layers):
        MLP.add(Dense(nodes, activation=activation_input))

    # Output layer (fully-connected):
    MLP.add(Dense(num_classes, activation='softmax'))
    MLP.compile(loss=categorical_crossentropy,
                optimizer='SGD',
                metrics='accuracy')
    return MLP

### Split+shuffle X and Y into k=num_folds different folds:
def KFold_split(X, Y, num_folds, seed):
    KFold_splitter = KFold(n_splits=num_folds, shuffle=True, random_state=seed)
    X_train_folds = []
    X_val_folds = []
    Y_train_folds = []
    Y_val_folds = []
    for (kth_fold_train_idxs, kth_fold_val_idxs) in KFold_splitter.split(X, Y):
        X_train_folds.append(X[kth_fold_train_idxs])
        X_val_folds.append(X[kth_fold_val_idxs])
        Y_train_folds.append(Y[kth_fold_train_idxs])
        Y_val_folds.append(Y[kth_fold_val_idxs])
    return X_train_folds, X_val_folds, Y_train_folds, Y_val_folds

#Data preprocessing

- display some meta information about the dataset
- display some of the images used for classification for a general impression of the data worked with

In [ ]:
#read the data from the csv
dataX = pd.read_csv("handwritten_digits_images.csv")
print("INPUT DATA")
dataX.info()
dataX = dataX.to_numpy()
X = dataX.reshape(dataX.shape[0], 28, 28);

print()
print("LABELS")
dataY = pd.read_csv("handwritten_digits_labels.csv")
dataY.info()
dataY = dataY.to_numpy()
y = dataY.reshape(-1,1)

num_classes = len(np.unique(y))

#scale the data, one-hot-encode the labels
X, y = preprocess_data(X, y, num_classes)

In [ ]:
#display a random selection of the images with their correct label

#reverse the one-hot encoding
y_integers = np.argmax(y, axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, y_integers, random_state=221, test_size=0.01, shuffle=True)

fig = plt.figure(figsize=(20, 20))
for i in range(len(X_test)):
    #each image gets a subplot
    ax = fig.add_subplot(28, 25, i + 1, xticks=[], yticks=[])
    ax.imshow(X_test[i], cmap='Greys')
    #add the correct label as text to the subplot
    ax.text(0, 8, str(Y_test[i]))

#Model Selection

- each type of classifier is tested on the validation data to find the best-performing parameters (automated)
- then the performance of this best performing configuration of each classfiers is compared to the others to find out which one the overall best performing classifier on the validation dataset is (manually)
- the same random states during train test split assure that the same data is used during all steps

##Basic Neural Networks

In [ ]:
#requires one-hot encoding
input_shape = np.shape(X)

#split into train and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=221, test_size=0.4, shuffle=True)
#extract train and validation folds:
X_train_folds, X_val_folds, Y_train_folds, Y_val_folds = KFold_split(X_train, Y_train, num_folds=3, seed=221)

#model selection parameters
train_batch_size = [500, 1000]
nodes = [392,784]
layers = [1,2,3]

configs = []
accuracies = []

for tbs in train_batch_size:
    for node in nodes:
        for layer in layers:
            fold_accuracies = []
            #test the configuration on three folds of the training data
            for X_train_fold, X_val_fold, Y_train_fold, Y_val_fold in zip(X_train_folds, X_val_folds, Y_train_folds, Y_val_folds):
                #setup the model with the current configuration
                model = build_MLP(num_classes, node, layer, activation_input='relu')
                #build and train the model
                model.build(input_shape)
                model.fit(X_train_fold, Y_train_fold, validation_split=0.3, batch_size=tbs, epochs=50, verbose=0)
                #evaluate
                test_results = model.evaluate(X_val_fold, Y_val_fold, verbose=0)
                fold_accuracies.append(test_results[1])
            #save the average accuracy achieved and the corresponding configuration
            configs.append((tbs, node, layer))
            print("======")
            print("Configuration: " + str((tbs, node, layer)))
            print("Accuracy: " + str(fold_accuracies) + " Average: " + str(mean(fold_accuracies)))
            accuracies.append(mean(fold_accuracies))

#print the best-performing configuration
max_config_indx = accuracies.index(max(accuracies))
print()
print("=============")
print("best-performing configuration:")
print(configs[max_config_indx])
print(max(accuracies))

##Decision Tree Classifier

In [ ]:
#flatten input, one-hot encoding 
X = X.reshape(69999,784)
#split into train and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=221, test_size=0.4, shuffle=True)
#extract train and validation folds:
X_train_folds, X_val_folds, Y_train_folds, Y_val_folds = KFold_split(X_train, Y_train, num_folds=3, seed=221)

#model selection parameters
criterions = ['gini', 'entropy']
splitters = ['best', 'random']

configs = []
accuracies = []

for crit in criterions:
    for split in splitters:
        fold_accuracies = []
        #test the configuration on three folds of the training data
        for X_train_fold, X_val_fold, Y_train_fold, Y_val_fold in zip(X_train_folds, X_val_folds, Y_train_folds, Y_val_folds):
            #setup the classifier with the current configuration
            clf = DecisionTreeClassifier(criterion=crit, splitter=split, random_state=0)
            #train
            clf.fit(X_train_fold, Y_train_fold)
            #predict
            Y_pred = clf.predict(X_val_fold)
            #evaluate
            accu = -1000 #in case something doesn't work (will be noticed during inspection of the results)
            accu = accuracy_score(Y_val_fold, Y_pred)
            fold_accuracies.append(accu)
        #save the average accuracy achieved and the corresponding configuration
        configs.append((crit, split))
        print("======")
        print("Configuration: " + str((crit, split)))
        print("Accuracy: " + str(fold_accuracies) + " Average: " + str(mean(fold_accuracies)))
        accuracies.append(mean(fold_accuracies))

#print the best-performing configuration
max_config_indx = accuracies.index(max(accuracies))
print()
print("=============")
print("best-performing configuration:")
print(configs[max_config_indx])
print(max(accuracies))

##kNN-Classifier

In [ ]:
#reverse one-hot-encoding, flatten input
X_flattened = X.reshape(69999,784)
y_integers = np.argmax(y, axis=1)
#split into train and test data
X_train, X_test, Y_train, Y_test = train_test_split(X_flattened, y_integers, random_state=221, test_size=0.4, shuffle=True)
#extract train and validation folds:
X_train_folds, X_val_folds, Y_train_folds, Y_val_folds = KFold_split(X_train, Y_train, num_folds=3, seed=221)

#model selection parameters
ks = [32, 64, 512, 1024]
weights = ['uniform', 'distance']

configs = []
accuracies = []

for k in ks:
    for w in weights:
        fold_accuracies = []
        #test the configuration on three folds of the training data
        for X_train_fold, X_val_fold, Y_train_fold, Y_val_fold in zip(X_train_folds, X_val_folds, Y_train_folds, Y_val_folds):
            #setup classifier with the current configuration
            k_NN = neighbors.KNeighborsClassifier(k, weights=w)
            #train
            k_NN.fit(X_train_fold,Y_train_fold) 
            #predict
            Y_pred = k_NN.predict(X_val_fold)
            #evaluate
            accu = -1000 #in case something doesn't work (will be noticed during inspection of the results)
            accu = accuracy_score(Y_val_fold, Y_pred)
            fold_accuracies.append(accu)
        #save the average accuracy achieved and the corresponding configuration
        configs.append((k,w)) 
        accuracies.append(mean(fold_accuracies))
        print("======")
        print("Configuration: " + str((k,w)))
        print("Accuracy: " + str(fold_accuracies) + " Average: " + str(mean(fold_accuracies)))

#print the best-performing configuration
max_config_indx = accuracies.index(max(accuracies))
print()
print("=============")
print("best-performing configuration:")
print(configs[max_config_indx])
print(max(accuracies))

## Support Vector Machine

In [ ]:
#reverse one-hot-encoding, flatten input
X_flattened = X.reshape(69999,784)
y_integers = np.argmax(y, axis=1)

#split into train and test data
X_train, X_test, Y_train, Y_test = train_test_split(X_flattened, y_integers, random_state=221, test_size=0.4, shuffle=True)
#extract train and validation folds:
X_train_folds, X_val_folds, Y_train_folds, Y_val_folds = KFold_split(X_train, Y_train, num_folds=3, seed=221)

#model selection parameters
kernels = ['linear', 'poly', 'rbf']
degrees = [2,3] #only significant in poly
coef0s = [0,1,2] #only significant in poly
gammas = ['scale', 'auto'] #ignored for linear kernel
C = [1,2,3]

configs = []
accuracies = []

for gamma in gammas:
    for kernel in kernels:
        for deg in degrees:
            for coef in coef0s:
                for c in C:
                    fold_accuracies = []
                    if kernel != 'poly' and deg != 2:
                        pass #degree is only significant for poly kernel
                    elif kernel != 'poly' and coef != 0:
                        pass #coef0 is only significant for poly kernel
                    elif kernel == 'linear' and gamma != 'auto': 
                        pass #gamma parameter is not significant for linear kernel
                    if gamma != 'auto' or coef != 2 or deg != 3 or kernel != 'poly' or c != 2:
                        pass
                    else:
                        #test the configuration on three folds of the training data
                        for X_train_fold, X_val_fold, Y_train_fold, Y_val_fold in zip(X_train_folds, X_val_folds, Y_train_folds, Y_val_folds):
                                #setup classifier with the current configuration
                                clf = SVC(decision_function_shape='ovo', kernel=kernel, gamma=gamma, coef0=coef, degree=deg, C=c)
                                #train
                                clf.fit(X_train_fold, Y_train_fold)
                                #predict
                                Y_pred = clf.predict(X_val_fold)
                                #evaluate
                                accu = -1000 #in case something doesn't work (will be noticed during inspection of the results)
                                accu = accuracy_score(Y_val_fold, Y_pred)
                                fold_accuracies.append(accu)
                        #save the average accuracy achieved and the corresponding configuration
                        configs.append((gamma, coef, deg, kernel, c))
                        print("======")
                        print("Configuration: " + str((gamma, coef, deg, kernel, c)))
                        print("Accuracy: " + str(fold_accuracies) + " Average: " + str(mean(fold_accuracies)))
                        accuracies.append(mean(fold_accuracies))

#print the best-performing configuration
max_config_indx = accuracies.index(max(accuracies))
print()
print("=============")
print("best-performing configuration:")
print(configs[max_config_indx])
print(max(accuracies))

#Evaluation

- below the best performing classifier from all of the above is evaluated on the test data

In [ ]:
#reverse one-hot-encoding, flatten input
X_flattened = X.reshape(69999,784)
y_integers = np.argmax(y, axis=1)

#split into train and test data
X_train, X_test, Y_train, Y_test = train_test_split(X_flattened, y_integers, random_state=221, test_size=0.4, shuffle=True)

#setup classifier with the best-performing configuration
clf = SVC(decision_function_shape='ovo', kernel='rbf', gamma='scale', C=3)
#train
clf.fit(X_train, Y_train)
#predict
Y_pred = clf.predict(X_test)
#evaluate
print("Accuracy: ", accuracy_score(Y_test, Y_pred))